# 0. Introdução

**Trabalho**:

Aluno: Maicon Dall'Agnol

R.A.: 151161868

Disciplina: Tópico em Aprendizado de Máquina

**Objetivos** :

- Escolha dois datasets rotulados.
- Realize a análise estatística, visualização e pré-processamento dos dados.
-  Realize os experimentos criando duas bases de teste distintas:
- - considerando todos os atributos do dataset;
- - selecionando alguns atributos e descartando outros.
- Aplique três métodos de classificação distintos nas duas bases acima referentes a cada dataset.
- Para cada dataset, em cada uma das bases, analise os resultados segundo medidas de qualidade de classificação, usando índices de validação externa (acurácia, recall, precisão, F-measure, índice Kappa) e cruva ROC.
- Proponha uma maneira adicional de comparar os resultados obtidos além das medidas acima.
- Compare e interprete os resultados dos dois experimentos em cada dataset.
- Faça tabela com as medidas de validação

## 0.1 Dependências

Para realização da tarefa foram utilizados as seguintes bibliotecas:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# KFold
from sklearn.model_selection import KFold

#Metricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from  sklearn.metrics  import  roc_auc_score 

from sklearn.metrics import balanced_accuracy_score

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# 1. Dados

Este conjunto de dados classifica as pessoas descritas por um conjunto de atributos como riscos de crédito bons ou ruins

## 1.1 Informações sobre os dados:

#### Atributos:
- Status da conta corrente existente, no marco alemão.
- Duração em meses
- Histórico de crédito (créditos recebidos, devolução devida, atrasos, contas críticas)
- Objetivo do crédito (carro, televisão, ...)
- Quantidade de crédito
- Situação da conta-poupança / títulos, no marco alemão.
- Emprego atual, em número de anos.
- Taxa de parcelamento em percentagem do rendimento disponível
- Estatuto pessoal (casado, solteiro, ...) e sexo
- Outros devedores / fiadores
- Residência atual desde X anos
- Propriedade (por exemplo, imóveis)
- Idade em anos
- Outros planos de parcelamento (bancos, lojas)
- Habitação (aluguel, próprio, ...)
- Número de créditos existentes neste banco
- Trabalho
- Número de pessoas susceptíveis de fornecer manutenção para
- Telefone (sim, não)
- trabalhador estrangeiro (sim, não)

#### Classe:
- Class



## Importando Dataset

In [5]:
data_news_raw = pd.read_json('News_Category_Dataset_v2.json', lines=True)

In [11]:
data_news_raw.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [15]:
data_news_df = data_news_raw.drop_duplicates()

In [20]:
pandas_profiling.ProfileReport(data_news_df)

Number of variables,7
Number of observations,200840
Total Missing (%),0.0%
Total size in memory,10.7 MiB
Average record size in memory,56.0 B
Numeric,1
Categorical,5
Boolean,0
Date,1
Text (Unique),0
Rejected,0


In [113]:
import re

In [289]:
www = re.compile(r'(https://www.|http://www.|http://|https://|http)')
dotcom = re.compile(r'(\.|\-)')

In [328]:
line

'https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89'

In [329]:
sites = []
for line in data_news_df.link:
    aux = line.split(sep = 'http', )
    try:
        aux = 'http' + aux[2]
    except:
        aux = line
    
    init = www.search(aux).span()[1]
    end = dotcom.search(aux, init+1).span()[0]
    
    sites.append(aux[init:end])

In [330]:
data_news_df['Site'] = sites

In [331]:
data_news_df.Site.unique()

array(['huffingtonpost', 'outsports', 'newnownext', ..., 'unioncity',
       'lightbox', 'searchengineland'], dtype=object)

In [332]:
data_news_df['Site'].value_counts()

huffingtonpost              190854
nytimes                        708
pubx                           609
washingtonpost                 303
politico                       218
thedodo                        192
theguardian                    175
refinery29                     169
blogs                          167
247wallst                      145
nymag                          130
realsimple                     120
buzzfeed                       120
insidemovies                   114
theatlantic                    108
thinkprogress                  105
cnn                             86
hollywoodreporter               78
mommyish                        77
slate                           74
forbes                          69
circleofmoms                    69
houzz                           68
newyorker                       68
bloomberg                       67
mnn                             66
online                          65
nbcnews                         63
greatist            

In [24]:
import nltk

In [25]:
sentence = "At eight o'clock on Thursday morning. Arthur didn't feel very good."
tokens = nltk.word_tokenize(sentence)
tokens

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/maicon/nltk_data'
    - '/home/maicon/anaconda3/nltk_data'
    - '/home/maicon/anaconda3/share/nltk_data'
    - '/home/maicon/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
